In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm


In [2]:
dfPlaylists = pd.read_csv("./playlistData/spotify_dataset.csv", on_bad_lines='skip')

In [3]:
dfPlaylists.head()

,user_id,"""artistname""","""trackname""","""playlistname"""
0,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,(The Angels Wanna Wear My) Red Shoes,HARD ROCK 2010
1,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,"(What's So Funny 'Bout) Peace, Love And Unders...",HARD ROCK 2010
2,9cc0cfd4d7d7885102480dd99e7a90d6,Tiffany Page,7 Years Too Late,HARD ROCK 2010
3,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,Accidents Will Happen,HARD ROCK 2010
4,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,Alison,HARD ROCK 2010


In [4]:
print(len(dfPlaylists))

12891680


In [5]:
playlists = []
playlistNames = []
dfPlaylists.iloc[0][3]
for i in range(20000): # change to the whole thing at end
    currentPLName = dfPlaylists.iloc[i][3]
    if (currentPLName not in playlistNames):
        playlistNames.append(currentPLName)
        playlists.append([])
        
    playlists[playlistNames.index(currentPLName)].append((dfPlaylists.iloc[i][1], dfPlaylists.iloc[i][2]))

In [6]:
playlistsOG = playlists.copy()
print(len(playlists))

251


In [7]:
countSingeltons = {}

for i in playlists:
    for j in i:
        if (j not in countSingeltons):
            countSingeltons[j] = 1
        else:
            countSingeltons[j] += 1

In [8]:
keys = countSingeltons.keys()
frequentSingeltons = {}
threshold = 4

for i in keys:
    if (countSingeltons[i] >= threshold):
        frequentSingeltons[i] = countSingeltons[i]

In [9]:
frequentSingeltonKeys = list(frequentSingeltons.keys())
print(frequentSingeltonKeys[0])

('Crosby, Stills & Nash', 'Southern Cross')


In [10]:
# trim playlists
def trimPlaylistSingelton(playlists, frequentKeys):
    trimPlaylists = []
    for i in playlists:
        valid = False
        for j in i:
            if (j in frequentKeys):
                valid = True

        if (valid):
            trimPlaylists.append(i)
    
    return trimPlaylists

preLength = len(playlists)
playlists = trimPlaylistSingelton(playlists, frequentSingeltonKeys)
        
print(preLength, len(playlists))

251 120


In [11]:
def matrixToTriangular(i, j, n):
    i = i + 1
    j = j + 1
    return int(((i - 1) * (n - (i / 2)) + j - i) - 1)

In [12]:
length = len(frequentSingeltonKeys)
maxIndex = matrixToTriangular(length - 2, length -1, length)
print(length, maxIndex)

243 29402


In [13]:
def countPairs(key1, key2, playlist):
    count = 0
    
    start = 0
    for i in playlist:
        if (key1 in i and key2 in i):
            count += 1

    return count

In [14]:
triangularCounts1 = [0 for i in range(maxIndex + 1)]

def fillTriangleCounts(length, keys, playlist, triangularCounts):
    for i in range(0, length): # ideally do this with the spotify id's to pass less string
        for j in range(i + 1, length):
            index = matrixToTriangular(i, j, length)
            triangularCounts[index] = countPairs(keys[i], keys[j], playlist)
            
fillTriangleCounts(length, frequentSingeltonKeys, playlists, triangularCounts1)


In [15]:
frequentPairs = {}
lengthTriangular = len(triangularCounts1)
for i in range(length):
    for j in range(i + 1, length):
        index = matrixToTriangular(i, j, length)
        if (triangularCounts1[index] >= threshold):
            frequentPairs[(frequentSingeltonKeys[i], frequentSingeltonKeys[j])] = triangularCounts1[index]
            
print(list(frequentPairs.keys())[0])
frequentPairKeys = list(frequentPairs.keys())

(('Daft Punk', 'Derezzed'), ('Daft Punk', 'Tron Legacy (End Titles)'))


In [16]:
recommendations = []
for i in range(2): # change to amount of playlists to run
    recForPlaylist = []
    for j in frequentPairKeys:
        first = j[0]
        second = j[1]
        
        for k in range(len(playlists[i])):
            if (playlists[i][k] == first):
                recForPlaylist.append(second)
            elif (playlists[i][k] == second):
                recForPlaylist.append(first)
                
    recommendations.append(recForPlaylist)

In [17]:
print(recommendations)

[[], []]


In [18]:
def trimPlaylistPairs(playlists, frequentKeys):
    trimPlaylists = []
    for i in playlists:
        isValidPlaylist = False
        for j in frequentKeys:
            isKeyItemsInPlaylist = [False for i in range(len(j))]
            for k in range(len(j)):
                if (j[k] in i):
                    isKeyItemsInPlaylist[k] = True
            if (False not in isKeyItemsInPlaylist):
                isValidPlaylist = True

        if (isValidPlaylist):
            trimPlaylists.append(i)
    
    return trimPlaylists

In [19]:
possibleTriples = []
#frequentPairKeys = [("a", "b"), ("b", "c"), ("a", "c"), ("b", "d"), ("a", "d"), ("a", "e"), ("b", "e")]

# it can only be a triple if {a, b} {b, c} and {a, c} are all frequent doubles
for i in range(len(frequentPairKeys)):
    a = frequentPairKeys[i][0]
    b = frequentPairKeys[i][1]
    c = None
    
    doesExistBC = False
    doesExistAC = False
    for j in range(i + 1, len(frequentPairKeys)):
        if (frequentPairKeys[j][0] == a):
            c = frequentPairKeys[j][1]
            doesExistAC = True
        if (frequentPairKeys[j][1] == a):
            c = frequentPairKeys[j][0]
            doesExistAC = True
            
            
        if (frequentPairKeys[j][0] == b):
            c = frequentPairKeys[j][1]
            doesExistBC = True
        if (frequentPairKeys[j][1] == b):
            c = frequentPairKeys[j][0]
            doesExistBC = True
            
        if (c != None):
            frequentSublist = frequentPairKeys[j: len(frequentPairKeys)]
            if (doesExistBC):
                AC = (a, c)
                if (AC in frequentSublist):
                    doesExistAC = True
                
            if (doesExistAC):
                BC = (b, c)
                if (BC in frequentSublist):
                    doesExistBC = True
            
        if (doesExistBC and doesExistAC):
            possibleTriples.append((a, b, c))
        
        c = None
        doesExistBC = False
        doesExistAC = False
        
print(len(possibleTriples))

1328


In [20]:
preLength = len(playlists)
playlistsNew = trimPlaylistPairs(playlists, frequentPairKeys)
print(preLength, len(playlistsNew))

length = len(frequentPairKeys)
maxIndex = matrixToTriangular(length - 2, length -1, length)
print(length, maxIndex)

120 48
447 99680


In [21]:
print(len(frequentSingeltonKeys))
print(len(frequentPairKeys))
print(len(possibleTriples))

243
447
1328


In [22]:
def countTriples(key1, key2, key3, playlist):
    count = 0

    for i in playlist:
        if (key1 in i and key2 in i and key3 in i):
            count += 1

    return count

In [23]:
frequentTriples = {}
for i in possibleTriples:
    count = countTriples(i[0], i[1], i[2], playlistsNew)
    if (count >= threshold):
        frequentTriples[i] = count
        
frequentTriplesKeys = list(frequentTriples.keys())
print(len(frequentTriplesKeys))
    

1206


In [24]:
possibleQuads = []

#frequentTriplesKeys = [("a", "b", "c"), ("a", "b", "d"), ("a", "c", "d"), ("b", "c", "d")]

# quad exists if {a, b, c}, {a, b, d}, {a, d, c}, and {d, b, c} exist
for i in range(len(frequentTriplesKeys)):
    a = frequentTriplesKeys[i][0]
    b = frequentTriplesKeys[i][1]
    c = frequentTriplesKeys[i][2]
    d = None
    
    doesExistABD = False
    doesExistADC = False
    doesExistBDC = False
    for j in range(i + 1, len(frequentTriplesKeys)):
        # Located {a, b, d}
        if (a in frequentTriplesKeys[j] and b in frequentTriplesKeys[j]):
            indexA, indexB = frequentTriplesKeys[j].index(a), frequentTriplesKeys[j].index(b)
            indexD = 3 - (indexA + indexB)
            d = frequentTriplesKeys[j][indexD]
            doesExistABD = True

        # Located {a, d, c}
        if (a in frequentTriplesKeys[j] and c in frequentTriplesKeys[j]):
            indexA, indexC = frequentTriplesKeys[j].index(a), frequentTriplesKeys[j].index(c)
            indexD = 3 - (indexA + indexC)
            d = frequentTriplesKeys[j][indexD]
            doesExistADC = True
            
        # Located {d, b, c}
        if (b in frequentTriplesKeys[j] and c in frequentTriplesKeys[j]):
            indexB, indexC = frequentTriplesKeys[j].index(b), frequentTriplesKeys[j].index(c)
            indexD = 3 - (indexC + indexB)
            d = frequentTriplesKeys[j][indexD]
            doesExistBDC = True
            
        if (d != None):
            frequentSublist = frequentTriplesKeys[j: len(frequentTriplesKeys)]
            # Looking for {a, d, c} {a, c, d} {d, a, c} {d, c, a} {c, a, d} {c, d, a} and 
            #             {b, d, c} {b, c, d} {c, b, d} {c, d, b} {d, c, b} {d, b, c}
            if (doesExistABD):
                ACD = (a, c, d) # the only way for {a, d, c} to appear is in order ACD (because of how the keys are stored)
                BCD = (b, c, d)
                if (ACD in frequentSublist):
                    doesExistADC = True
                if (BCD in frequentSublist):
                    doesExistBDC = True

            if (doesExistADC):
                ABD = (a, b, d)
                BCD = (b, c, d)
                
                if (ABD in frequentSublist):
                    doesExistABD = True
                if (BCD in frequentSublist):
                    doesExistBDC = True
                    
            if (doesExistBDC):
                ADC = (a, c, d)
                ABD = (a, b, d)
                
                if (ADC in frequentSublist):
                    doesExistADC = True
                if (ABD in frequentSublist):
                    doesExistABD = True
                
                
        if (doesExistABD and doesExistADC and doesExistBDC):
            possibleQuads.append((a, b, c, d))
        
        d = None
        doesExistABD = False
        doesExistADC = False
        doesExistBDC = False
        
print(len(possibleQuads))

3060


In [25]:
for i in range(len(possibleQuads)):
    a = possibleQuads[0]
    b = possibleQuads[1]
    c = possibleQuads[2]
    d = possibleQuads[3]
    
    for j in range(i + 1, len(possibleQuads)):
        if ((a in possibleQuads[j]) and (b in possibleQuads[j]) and (c in possibleQuads[j]) and (d in possibleQuads[j])):
            print(a, b, c, d)

In [26]:
print(possibleQuads[0])
print(len(playlistsNew))

(('Pendulum', '9,000 Miles'), ('Crystal Castles', 'Celestica'), ('Justice', 'D.A.N.C.E - MSTRKRFT Remix'), ('Justice', 'Genesis'))
48


In [27]:
def countQuads(key1, key2, key3, key4, playlist):
    count = 0

    for i in playlist:
        if (key1 in i and key2 in i and key3 in i and key4 in i):
            count += 1

    return count

In [28]:
frequentQuads = {}
for i in possibleQuads:
    count = countQuads(i[0], i[1], i[2], i[3], playlistsNew)
    if (count >= threshold):
        frequentQuads[i] = count
        
frequentQuadsKeys = list(frequentQuads.keys())
print(len(frequentQuadsKeys))

3053


In [36]:
recommendations = []
for i in range(10): # change to amount of playlists to run
    recForPlaylist = []
    for j in frequentQuadsKeys:
        a = j[0]
        b = j[1]
        c = j[2]
        d = j[3]
        
        if (a in playlistsOG[i]):
            if (b not in playlistsOG[i] and b not in recForPlaylist):
                recForPlaylist.append(b)
            if (c not in playlistsOG[i] and c not in recForPlaylist):
                recForPlaylist.append(c)
            if (d not in playlistsOG[i] and d not in recForPlaylist):
                recForPlaylist.append(d)
            
        if (b in playlistsOG[i]):
            if (a not in playlistsOG[i] and a not in recForPlaylist):
                recForPlaylist.append(a)
            if (c not in playlistsOG[i] and c not in recForPlaylist):
                recForPlaylist.append(c)
            if (d not in playlistsOG[i] and d not in recForPlaylist):
                recForPlaylist.append(d)

        if (c in playlistsOG[i]):
            if (b not in playlistsOG[i] and b not in recForPlaylist):
                recForPlaylist.append(b)
            if (a not in playlistsOG[i] and a not in recForPlaylist):
                recForPlaylist.append(a)
            if (d not in playlistsOG[i] and d not in recForPlaylist):
                recForPlaylist.append(d)
            
        if (d in playlistsOG[i]):
            if (b not in playlistsOG[i] and b not in recForPlaylist):
                recForPlaylist.append(b)
            if (c not in playlistsOG[i] and c not in recForPlaylist):
                recForPlaylist.append(c)
            if (a not in playlistsOG[i] and a not in recForPlaylist):
                recForPlaylist.append(a)
                
    recommendations.append(recForPlaylist)
    

print(recommendations[7])
print()
print(playlistsOG[7])

[('C2C', 'Down The Road'), ('Korn', 'Narcissistic Cannibal (feat. Skrillex & Kill The Noise)'), ('The Offspring', "The Kids Aren't Alright")]

[('Crystal Castles', '1991'), ('SomethingALaMode', '5 AM'), ('Pendulum', '9,000 Miles'), ('Data', 'Aerius Light (Kitsuné Dj-Friendly Edit)'), ('Crystal Castles', 'Air War - David Wolf Remix'), ('Crystal Castles', 'Alice Practice'), ('Ratatat', 'Alps'), ('Crystal Castles', 'Baptism'), ('Ratatat', 'Bare Feast'), ('Ratatat', 'Bilar'), ('Ratatat', 'Bird-Priest'), ('Ratatat', 'Black Heroes'), ('Ratatat', 'Bob Gandhi'), ('Ratatat', 'Breaking Away'), ('Ratatat', 'Bruleé'), ('Ratatat', 'Bustelo'), ('Crystal Castles', 'Celestica'), ('Ratatat', 'Cherry'), ('The Prodigy', 'Colours'), ('Crystal Castles VS Health', 'Crimewave'), ('Ratatat', 'Crips'), ('Justice', 'D.A.N.C.E'), ('Justice', 'D.A.N.C.E - MSTRKRFT Remix'), ('Justice', 'D.A.N.C.E. - Radio Edit'), ('Justice', 'DVNO'), ('Ratatat', 'Desert Eagle'), ('Pendulum', 'Different'), ('Glass Candy', 'Digital 

In [30]:
maximalQuads = dict(frequentQuads)
maximalTriples = frequentTriplesKeys.copy()
maximalPairs = frequentPairKeys.copy()

print(len(maximalQuads))
print(len(maximalTriples))
for i in frequentTriplesKeys:
    a = i[0]
    b = i[1]
    c = i[2]
    
    for j in frequentQuadsKeys:
        if (a in j and b in j and c in j):
            index = maximalTriples.index(i)
            maximalTriples.pop(index)
            break
        
print(len(maximalTriples))

print(len(maximalPairs))
for i in frequentPairKeys:
    a = i[0]
    b = i[1]
    
    for j in frequentQuadsKeys:
        if (a in j and b in j):
            index = maximalPairs.index(i)
            maximalPairs.pop(index)
            break

print(len(maximalPairs))
maxPairsIntermediate = maximalPairs.copy()
for i in maxPairsIntermediate:
    a = i[0]
    b = i[1]
    for j in maximalTriples:
        if (a in j and b in j):
            index = maximalPairs.index(i)
            maximalPairs.pop(index)
            break
    
print(len(maximalPairs))

3053
1206
25
447
77
36


In [31]:
maximal = maximalQuads.copy()
for i in maximalTriples:
    maximal[i] = frequentTriples[i]
    
for i in maximalPairs:
    maximal[i] = frequentPairs[i]
    
maximalMatrix = []
for i in maximal.keys():
    vectorMaximal = []
    if (len(i) == 4):
        vectorMaximal.append(i[0])
        vectorMaximal.append(i[1])
        vectorMaximal.append(i[2])
        vectorMaximal.append(i[3])
    
    if (len(i) == 3):
        vectorMaximal.append(i[0])
        vectorMaximal.append(i[1])
        vectorMaximal.append(i[2])
        vectorMaximal.append(None)
    
    if (len(i) == 2):
        vectorMaximal.append(i[0])
        vectorMaximal.append(i[1])
        vectorMaximal.append(None)
        vectorMaximal.append(None)
        
    maximalMatrix.append(vectorMaximal)

In [32]:
# (1, 2, 3, 4) = {1: (2, 3, 4) -> 1}, {2: (1, 3, 4) -> 2}, {3: (1, 2, 4) -> 3}, {4: (1, 2, 3) -> 4}, 
#      (1, 2, 3) { 5: (2, 3) -> 1}, { 6: (1, 3) -> 2}, { 7: (1, 2) -> 3},
#      (1, 2, 4) { 8: (2, 4) -> 1}, { 9: (1, 4) -> 2}, {10: (1, 2) -> 4},
#      (1, 3, 4) {11: (3, 4) -> 1}, {12: (1, 4) -> 3}, {13: (1, 3) -> 4},
#      (2, 3, 4) {14: (3, 4) -> 2}, {15: (2, 4) -> 3}, {16: (2, 3) -> 4},
#             (1, 2) {17: (2) -> 1}, {18: (1) -> 2},
#             (1, 3) {19: (3) -> 1}, {20: (1) -> 3},
#             (1, 4) {21: (4) -> 1}, {22: (1) -> 4},
#             (2, 3) {23: (3) -> 2}, {24: (2) -> 3},
#             (2, 4) {25: (4) -> 2}, {26: (2) -> 4},
#             (3, 4) {27: (4) -> 3}, {28: (3) -> 4}

# (1, 2, 3) = {1: (2, 3) -> 1}, {2: (1, 3) -> 2}, {3: (1, 2) -> 3}
#      (1, 2) {4: (2) -> 1}, {5: (1) -> 2},
#      (1, 3) {6: (3) -> 1}, {7: (1) -> 3},
#      (2, 3) {8: (3) -> 2}, {9: (2) -> 3}

# (1, 2) = {1: (2) -> 1}, {2: (1) -> 2}
intrestMatrix = []
count = 0
for i in maximalMatrix:
    intrestMaximal = []
    i = maximalMatrix[count].copy()
    if (None in i):
        i.remove(None)
        if (None in i):
            i.remove(None)
            
    if (len(i) == 4):
        countFull = frequentQuads[tuple(i)]
        for j in i:
            subsection = i.copy()
            subsection.remove(j)
            countj = countSingeltons[j]
            countSubsection = frequentTriples[tuple(subsection)]
            confidence = countFull / countSubsection
            support = countj / len(playlists)
            intrest = confidence - support
            intrestMaximal.append(intrest)
            
        for k in range(4):
            if (k == 0):
                sub1 = [i[0], i[1], i[2]]
            elif (k == 1):
                sub1 = [i[0], i[1], i[3]]
            elif (k == 2):
                sub1 = [i[0], i[2], i[3]]
            elif (k == 3):
                sub1 = [i[1], i[2], i[3]]
                
            countFull = frequentTriples[tuple(sub1)]
            for j in sub1:
                subsection = sub1.copy()
                subsection.remove(j)
                countj = countSingeltons[j]
                countSubsection = frequentPairs[tuple(subsection)]
                confidence = countFull / countSubsection
                support = countj / len(playlists)
                intrest = confidence - support
                intrestMaximal.append(intrest)
                
        for k in range(6):
            if (k == 0):
                sub2 = [i[0], i[1]]
            elif (k == 1):
                sub2 = [i[0], i[2]]
            elif (k == 2):
                sub2 = [i[0], i[3]]
            elif (k == 3):
                sub2 = [i[1], i[2]]
            elif (k == 4):
                sub2 = [i[1], i[3]]
            elif (k == 5):
                sub2 = [i[2], i[3]]
                
            countFull = frequentPairs[tuple(sub2)]
            for j in sub2:
                subsection = sub2.copy()
                subsection.remove(j)
                subsection = subsection[0]
                countj = countSingeltons[j]
                countSubsection = frequentSingeltons[tuple(subsection)]
                confidence = countFull / countSubsection
                support = countj / len(playlists)
                intrest = confidence - support
                intrestMaximal.append(intrest)
                
            
    elif (len(i) == 3):
        countFull = frequentTriples[tuple(i[:3])]
        for j in i:
            subsection = i.copy()
            subsection.remove(j)
            countj = countSingeltons[j]
            countSubsection = frequentPairs[tuple(subsection)]
            confidence = countFull / countSubsection
            support = countj / len(playlists)
            intrest = confidence - support
            intrestMaximal.append(intrest)
        
        for k in range(3):
            if (k == 0):
                sub2 = [i[0], i[1]]
            elif (k == 1):
                sub2 = [i[0], i[2]]
            elif (k == 2):
                sub2 = [i[1], i[2]]

            countFull = frequentPairs[tuple(sub2)]
            for j in sub2:
                subsection = sub2.copy()
                subsection.remove(j)
                subsection = subsection[0]
                countj = countSingeltons[j]
                countSubsection = frequentSingeltons[tuple(subsection)]
                confidence = countFull / countSubsection
                support = countj / len(playlists)
                intrest = confidence - support
                intrestMaximal.append(intrest)
                
        for j in range(19):
            intrestMaximal.append(None)
        
    elif(len(i) == 2):
        countFull = frequentPairs[tuple(i[:3])]
        for j in i:
            subsection = i.copy()
            subsection.remove(j)
            subsection = subsection[0]
            countj = countSingeltons[j]
            countSubsection = frequentSingeltons[tuple(subsection)]
            confidence = countFull / countSubsection
            support = countj / len(playlists)
            intrest = confidence - support
            intrestMaximal.append(intrest)
        
        for j in range(24):
            intrestMaximal.append(None)
    
    intrestMatrix.append(intrestMaximal)
    count +=1

In [33]:
for i in range(len(intrestMatrix)):
    for j in intrestMatrix[i]:
        maximalMatrix[i].append(j)

In [34]:
maximalsDF = pd.DataFrame(maximalMatrix)
maximalsDF.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,"(Pendulum, 9,000 Miles)","(Crystal Castles, Celestica)","(Justice, D.A.N.C.E - MSTRKRFT Remix)","(Justice, Genesis)",0.966667,0.958333,0.966667,0.966667,0.966667,0.958333,...,0.966667,0.966667,0.966667,0.966667,0.958333,0.766667,0.958333,0.766667,0.966667,0.966667
1,"(Pendulum, 9,000 Miles)","(Crystal Castles, Celestica)","(Justice, D.A.N.C.E - MSTRKRFT Remix)","(The Prodigy, Invaders Must Die)",0.966667,0.958333,0.966667,0.966667,0.966667,0.958333,...,0.966667,0.966667,0.966667,0.966667,0.958333,0.766667,0.958333,0.766667,0.966667,0.966667
2,"(Pendulum, 9,000 Miles)","(Crystal Castles, Celestica)","(Justice, D.A.N.C.E - MSTRKRFT Remix)","(Crystal Castles, Magic Spells)",0.966667,0.958333,0.966667,0.966667,0.966667,0.958333,...,0.966667,0.966667,0.966667,0.966667,0.958333,0.766667,0.958333,0.766667,0.966667,0.966667
3,"(Pendulum, 9,000 Miles)","(Crystal Castles, Celestica)","(Justice, D.A.N.C.E - MSTRKRFT Remix)","(Pendulum, Mutiny)",0.966667,0.958333,0.966667,0.966667,0.966667,0.958333,...,0.966667,0.966667,0.966667,0.966667,0.958333,0.766667,0.958333,0.766667,0.966667,0.966667
4,"(Pendulum, 9,000 Miles)","(Crystal Castles, Celestica)","(Justice, D.A.N.C.E - MSTRKRFT Remix)","(Ratatat, Nostrand)",0.966667,0.958333,0.966667,0.966667,0.966667,0.958333,...,0.966667,0.966667,0.966667,0.966667,0.958333,0.766667,0.958333,0.766667,0.966667,0.966667


In [35]:
maximalsDF.to_csv("./playlistData/maximal_T4_20.csv")